In [134]:
import os
import pandas as pd
import numpy as np
import csv
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from string import punctuation
import nltk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import f1_score, make_scorer
from string import punctuation
from operator import itemgetter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import word_tokenize, pos_tag, pos_tag_sents
ps = PorterStemmer()
import string 
from collections import Counter
from nltk import word_tokenize, pos_tag, pos_tag_sents
import matplotlib.pyplot as plt
import re
from imblearn.over_sampling import RandomOverSampler


In [65]:
data = pd.read_csv("train_file.csv")
data1 = pd.read_csv("test_file.csv")
data_temp1 = pd.concat([data,data1],axis=0)
data_temp1['Description'].fillna('No address',inplace= True)
data_temp1.shape
data_temp1['POS'] = pos_tag_sents(data_temp1['Description'].apply(word_tokenize).tolist())
data_temp1['noun_words'] = np.nan
data_temp1['noun_words_sentec'] = np.nan
data_temp1 = data_temp1.astype('object')

(55899, 19)

In [109]:
for j in range(len(data_temp1)):
    tagged = data_temp1.iloc[j,20]
    noun_words = []
    for item in tagged:
        if item[1] == 'N' or item[1] == 'NN' or item[1] == 'NNP' or item[1] == 'NNS' or item[1] == 'NNPS':
            noun_words.append(item[0])
    data_temp1.iloc[j,21] = noun_words

In [111]:
for j in range(len(data_temp1)):
    temp = ' '.join(word for word in data_temp1.iloc[j,21])
    data_temp1.iloc[j,22] = temp

In [112]:
no_features = 1000 #top 1000 features are selected
tf_vectorizer = CountVectorizer(max_df=0.95,min_df=0.02,stop_words='english')
tf = tf_vectorizer.fit_transform(data_temp1['noun_words_sentec'])
tf_feature_names = tf_vectorizer.get_feature_names()

c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [ ]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(tf)

In [136]:
ros = RandomOverSampler(random_state=9)
classifier = RandomForestClassifier(n_estimators=10)
ntopics = [5,10,20,40]
fscore = []
for i in range(len(ntopics)):
    lda_model = LatentDirichletAllocation(n_topics=ntopics[i],               # Number of topics
                                          max_iter=10,               # Max learning iterations
                                          learning_method='online',   
                                          random_state=100,          # Random state
                                          batch_size=128,            # n docs in each learning iter
                                          evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                          n_jobs = -1,               # Use all available CPUs
                                         )
    lda_output = lda_model.fit_transform(tf)
    Topic_distribution = DataFrame(data=lda_output)
    X_sample_0, y_sample_0 = ros.fit_sample(Topic_distribution.iloc[0:len(data)], data['Category'])
    X_train_O,X_test_O,y_train_O,y_test_O = train_test_split(X_sample_0,y_sample_0,test_size = 0.3)
    classifier.fit(X_train_O,y_train_O)
    y_pred = classifier.predict(X_test_O)
    fscore.append(f1_score(y_test_O, y_pred, average='weighted'))

c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  Depr

In [138]:
lda_model = LatentDirichletAllocation(n_topics=10,               # Number of topics
                                          max_iter=10,               # Max learning iterations
                                          learning_method='online',   
                                          random_state=100,          # Random state
                                          batch_size=128,            # n docs in each learning iter
                                          evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                          n_jobs = -1,               # Use all available CPUs
                                         )
lda_output = lda_model.fit_transform(tf)
Topic_distribution = DataFrame(data=lda_output)
Topic_distribution.to_csv("Topic_distribution.csv")

c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [141]:
X_sample_0, y_sample_0 = ros.fit_sample(Topic_distribution.iloc[0:len(data)], data['Category'])
classifier.fit(X_sample_0,y_sample_0)
y_pred = classifier.predict(Topic_distribution.iloc[len(data):len(data)+len(data1)])

In [142]:
Topic_prediction = pd.concat([data1['Application/Permit Number'],pd.DataFrame({'Category':y_pred})],axis=1)
Topic_prediction.to_csv('prediction3.csv', sep=',',index = False)

In [152]:
Topic_distribution.rename(columns={0:'Topic0',1:'Topic1',2:'Topic2',
                                  3:'Topic3',4:'Topic4',5:'Topic5',
                                  6:'Topic6',7:'Topic7',8:'Topic8',
                                  9:'Topic9'}, inplace = True)

In [155]:
Topic_distribution.to_csv("Topic_distribution.csv")